# Site Performance Data Preparation
This notebook loads and processes clinical trial site data from the AACT dataset.

In [1]:
import pandas as pd
from pathlib import Path

## Define Raw Data Directory

In [2]:
# Define your raw data folder
DATA_DIR = Path("../data/raw/aact-data")
print(f"Data directory: {DATA_DIR}")

Data directory: ../data/raw/aact-data


## Load Tables
Define and load AACT tables: `studies`, `facilities`, `sponsors`, and `conditions`.

In [3]:
# Tables to load
TABLES = {
    "studies": "studies.txt",
    "facilities": "facilities.txt",
    "sponsors": "sponsors.txt",
    "conditions": "conditions.txt"
}

# Load tables into DataFrames
dfs = {}
for name, filename in TABLES.items():
    file_path = DATA_DIR / filename
    print(f"Loading {name}...")
    dfs[name] = pd.read_csv(file_path, delimiter="|", dtype=str, low_memory=False)

Loading studies...
Loading facilities...
Loading sponsors...
Loading conditions...


## Extract and Subset DataFrames
Select only the relevant columns.

In [4]:
# Extract individual DataFrames
studies = dfs["studies"]
facilities = dfs["facilities"]
sponsors = dfs["sponsors"]
conditions = dfs["conditions"]

# Only keep relevant columns from each
studies_subset = studies[[
    "nct_id", "study_type", "phase", "enrollment", "overall_status", "start_date", "completion_date"
]]

facilities_subset = facilities[[
    "nct_id", "status", "name", "city", "state", "country"
]]

sponsors_subset = sponsors[sponsors["lead_or_collaborator"] == "lead"]
sponsors_subset = sponsors_subset[["nct_id", "agency_class"]]

# Optional: count number of conditions per study
condition_counts = conditions.groupby("nct_id").size().reset_index(name="condition_count")

## Merge Tables
Combine the extracted datasets into a single DataFrame.

In [5]:
# Merge tables together
merged = facilities_subset.merge(studies_subset, on="nct_id", how="left")
merged = merged.merge(sponsors_subset, on="nct_id", how="left")
merged = merged.merge(condition_counts, on="nct_id", how="left")

## Handle Missing Data and Feature Engineering

In [6]:
# Fill missing data
merged["enrollment"] = pd.to_numeric(merged["enrollment"], errors="coerce")
merged["condition_count"] = merged["condition_count"].fillna(0).astype(int)

# Feature: number of sites per study
site_counts = facilities.groupby("nct_id").size().reset_index(name="site_count")
merged = merged.merge(site_counts, on="nct_id", how="left")

# Feature: site success label
merged["site_success"] = merged["status"].apply(
    lambda x: 1 if str(x).lower() in ["active, not recruiting", "recruiting", "completed", "enrolling by invitation"] else 0
)

# Optional cleanup of category fields
merged["phase"] = merged["phase"].fillna("Unknown")
merged["study_type"] = merged["study_type"].fillna("Unknown")
merged["agency_class"] = merged["agency_class"].fillna("Unknown")
merged["country"] = merged["country"].fillna("Unknown")

## Create Final Modeling Dataset
Subset final features and preview the dataset.

In [7]:
# Final dataset
model_df = merged[[
    "nct_id", "name", "city", "state", "country", "study_type", "phase",
    "enrollment", "overall_status", "agency_class", "condition_count",
    "site_count", "site_success"
]]

print(f"Final modeling dataset shape: {model_df.shape}")
print(model_df["site_success"].value_counts())

Final modeling dataset shape: (3285407, 13)
site_success
0    2877330
1     408077
Name: count, dtype: int64


## Save Processed Dataset to CSV

In [8]:
# Save to CSV
OUTPUT_PATH = DATA_DIR.parent.parent / "site_performance.csv"
model_df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved final dataset to: {OUTPUT_PATH}")

Saved final dataset to: ../data/site_performance.csv
